In [8]:
import torch

In [1]:
!wget -P /opt/ml/input/data/sample/ https://drive.google.com/u/0/uc?id=1EjSfLv-eb-nWAW-kZuJOyWQFOy0uY1aE&export=download

In [30]:
torch.empty(size=(4,), device='cuda')

tensor([1.3873e-43, 0.0000e+00, 0.0000e+00, 0.0000e+00], device='cuda:0')

In [31]:
torch.empty(size=(3, 4), device='cuda')

tensor([[1.3873e-43, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00]], device='cuda:0')

In [32]:
torch.empty(size=(2, 3, 4), device='cuda')

tensor([[[1.3873e-43, 0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00]],

        [[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00]]], device='cuda:0')

In [33]:
torch.empty(size=(2, 3, 4), device='cuda')\
.fill_(0.1 / 3)

tensor([[[0.0333, 0.0333, 0.0333, 0.0333],
         [0.0333, 0.0333, 0.0333, 0.0333],
         [0.0333, 0.0333, 0.0333, 0.0333]],

        [[0.0333, 0.0333, 0.0333, 0.0333],
         [0.0333, 0.0333, 0.0333, 0.0333],
         [0.0333, 0.0333, 0.0333, 0.0333]]], device='cuda:0')

In [40]:
torch.empty(size=(2, 3, 4), device='cuda')\
.fill_(0.1 / 3)\
.scatter_(-1, torch.randint(1, 7, (4,)).data.unsqueeze(-1), torch.empty(size=(4,), device='cuda').fill_(0.9))

RuntimeError: Expected object of device type cuda but got device type cpu for argument #3 'index' in call to _th_scatter_

In [36]:
torch.randint(1, 7, (4,)).data.unsqueeze(-1)

tensor([[1],
        [1],
        [6],
        [5]])

In [2]:
!sh satrn12.sh

--------------------------------
Running SATRN on device cuda

[+] System environments
 The number of gpus : 1
 The number of cpus : 8
 Memory Size : 83G

[+] Data
 The number of train samples : 80000
 The number of validation samples : 20000
 The number of classes : 245

[+] Network
 Type: SATRN
 Encoder parameters: 5875140
 Decoder parameters: 988917 

teacher_forcing 0.5
Traceback (most recent call last):                                              
  File "train.py", line 475, in <module>
    main(parser.config_file)
  File "train.py", line 340, in main
    train=True,
  File "train.py", line 110, in run_epoch
    loss = criterion(decoded_values, expected[:, 1:])
  File "/opt/conda/lib/python3.7/site-packages/torch/nn/modules/module.py", line 532, in __call__
    result = self.forward(*input, **kwargs)
  File "/opt/ml/code/networks/SATRN.py", line 63, in forward
    targets = LabelSmoothingCrossEntropy._smooth_one_hot(targets, self.num_classes, self.smoothing, self.ignore_index) #

In [ ]:
# class LabelSmoothingLoss(nn.Module):
#     """
#     With label smoothing,
#     KL-divergence between q_{smoothed ground truth prob.}(w)
#     and p_{prob. computed by model}(w) is minimized.
#     """
#     def __init__(self, label_smoothing, tgt_vocab_size, ignore_index=-100):
#         assert 0.0 < label_smoothing <= 1.0
#         self.ignore_index = ignore_index
#         super(LabelSmoothingLoss, self).__init__()

#         smoothing_value = label_smoothing / (tgt_vocab_size - 2)
#         one_hot = torch.full((tgt_vocab_size,), smoothing_value) 
#         one_hot[self.ignore_index] = 0
#         self.register_buffer('one_hot', one_hot.unsqueeze(0))

#         self.confidence = 1.0 - label_smoothing

#     def forward(self, output, target):
#         """
#         output (FloatTensor): batch_size x n_classes
#         target (LongTensor): batch_size
#         """
#         model_prob = self.one_hot.repeat(target.size(0), 1)
#         model_prob.scatter_(1, target.unsqueeze(1), self.confidence)
#         model_prob.masked_fill_((target == self.ignore_index).unsqueeze(1), 0)

#         return F.kl_div(output, model_prob, reduction='sum')

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import math
import random

# 단일 Classification

In [14]:
# 사용될 변수 선언
smoothing_value = 0.1
vocab_size = 3
sentence_size = 5
ignore_index=0
batch_size = 2

In [15]:
temp = torch.full((tgt_vocab_size,), smoothing_value / (tgt_vocab_size - 2))

In [16]:
# 임의로 설정한 target
target = torch.rand(size=(batch_size, vocab_size, sentence_size))
type(target), target

(torch.Tensor,
 tensor([[[0.3275, 0.6315, 0.4642, 0.7919, 0.0574],
          [0.9240, 0.0242, 0.1589, 0.1785, 0.8554],
          [0.5115, 0.9508, 0.7066, 0.3282, 0.9478]],
 
         [[0.1753, 0.7012, 0.1265, 0.6870, 0.9951],
          [0.4266, 0.8570, 0.1130, 0.0105, 0.3365],
          [0.8641, 0.5406, 0.6147, 0.8150, 0.9605]]]))

In [5]:
# one_hot vector에 smoothing 적용
one_hot = torch.full((tgt_vocab_size,), smoothing_value/(7-2)) # size 만큼 smoothing_value로 채우기
one_hot[[0,1,2]] = 0.0
one_hot = one_hot.repeat(batch_size, 1)\
                .scatter(1, target.unsqueeze(1), 1 - smoothing_value)
one_hot

NameError: name 'target' is not defined

In [162]:
# 첫 번째 class는 사용X
one_hot[ignore_index] = 0.0
one_hot

tensor([0.0000, 0.0200, 0.0200, 0.0200, 0.0200, 0.0200, 0.0200])

In [163]:
# batch_size만큼 늘려주기
one_hot_smoothing = one_hot.repeat(batch_size, 1) # size: (*origin) => (4, *origin)
one_hot_smoothing

tensor([[0.0000, 0.0200, 0.0200, 0.0200, 0.0200, 0.0200, 0.0200],
        [0.0000, 0.0200, 0.0200, 0.0200, 0.0200, 0.0200, 0.0200],
        [0.0000, 0.0200, 0.0200, 0.0200, 0.0200, 0.0200, 0.0200],
        [0.0000, 0.0200, 0.0200, 0.0200, 0.0200, 0.0200, 0.0200]])

In [164]:
one_hot_smoothing.shape

torch.Size([4, 7])

In [6]:
# 임의로 설정한 target
target = torch.randint(1, 7, (batch_size,))
type(target), target

(torch.Tensor, tensor([6, 4, 3, 3]))

In [187]:
one_hot_smoothing.shape, target.shape, target.unsqueeze(-1).shape, target.unsqueeze(1).shape

(torch.Size([4, 7]), torch.Size([4]), torch.Size([4, 1]), torch.Size([4, 1]))

In [188]:
# one_hot_smoothing 완성
one_hot_smoothing = one_hot_smoothing.scatter(1, target.unsqueeze(1), 1 - smoothing_value) # scatter에서 inplace=True 옵션
one_hot_smoothing

tensor([[0.0000, 0.0200, 0.9000, 0.0200, 0.0200, 0.0200, 0.0200],
        [0.0000, 0.9000, 0.0200, 0.0200, 0.0200, 0.0200, 0.0200],
        [0.0000, 0.0200, 0.0200, 0.9000, 0.0200, 0.0200, 0.0200],
        [0.0000, 0.0200, 0.0200, 0.0200, 0.9000, 0.0200, 0.0200]])

In [189]:
one_hot_smoothing.shape

torch.Size([4, 7])

In [155]:
input = torch.rand((batch_size, tgt_vocab_size))
input

tensor([[0.0350, 0.1968, 0.1322, 0.4622, 0.0382, 0.0231, 0.5226],
        [0.8962, 0.8766, 0.5369, 0.0406, 0.3827, 0.1069, 0.7799],
        [0.8783, 0.7525, 0.5823, 0.6227, 0.3112, 0.1279, 0.0992],
        [0.4562, 0.2876, 0.9744, 0.1390, 0.7229, 0.7142, 0.6335]])

In [156]:
lsm = F.log_softmax(input, -1)
lsm

tensor([[-2.1318, -1.9700, -2.0346, -1.7046, -2.1287, -2.1437, -1.6443],
        [-1.6187, -1.6383, -1.9780, -2.4743, -2.1322, -2.4080, -1.7350],
        [-1.5888, -1.7147, -1.8849, -1.8445, -2.1559, -2.3393, -2.3680],
        [-2.0852, -2.2538, -1.5669, -2.4024, -1.8185, -1.8272, -1.9079]])

In [165]:
lsm.shape

torch.Size([4, 7])

In [157]:
loss = -(one_hot_smoothing2 * lsm).sum(-1)
loss

tensor([2.0088, 2.1560, 1.9011, 2.3914])

In [166]:
loss.sum(), loss.mean()

(tensor(8.4572), tensor(2.1143))

In [196]:
from torch.nn.modules.loss import _WeightedLoss

class LabelSmoothingCrossEntropy(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.1):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction

    @staticmethod
    def _smooth_one_hot(targets: torch.Tensor, n_classes: int, smoothing=0.0):
        assert 0 <= smoothing < 1
        with torch.no_grad():
#             targets = torch.empty(size=(targets.size(0), n_classes),
#                                   device=targets.device) \
#                 .fill_(smoothing / (n_classes - 1)) \
#                 .scatter_(-1, targets.data.unsqueeze(-1), 1. - smoothing) # dim=-1로 수정
            
            targets = torch.full((tgt_vocab_size,), smoothing_value/(7-2)) # size 만큼 smoothing_value로 채우기
            targets[[0,1,2]] = 0.0
            targests = targets.repeat(batch_size, 1)\
                            .scatter(1, target.unsqueeze(1), 1 - smoothing_value)
        return targets
    
            

    def forward(self, inputs, targets):
        targets = LabelSmoothingCrossEntropy._smooth_one_hot(targets, inputs.size(-1), self.smoothing)
        lsm = F.log_softmax(inputs, -1)

        if self.weight is not None:
            lsm = lsm * self.weight.unsqueeze(0)

        loss = -(targets * lsm).sum(-1)

        if self.reduction == 'sum':
            loss = loss.sum()
        elif self.reduction == 'mean':
            loss = loss.mean()

        return loss

In [159]:
one_hot_smoothing2 = torch.empty(size=(batch_size, tgt_vocab_size))
one_hot_smoothing2

tensor([[ 1.4013e-45,  0.0000e+00,  6.9868e-35,  3.0775e-41,  3.9236e-44,
          3.0775e-41,  2.9446e-22],
        [ 4.5766e-41,  2.8026e-45,  4.5766e-41,  2.8026e-45,  4.5766e-41,
          1.4013e-45,  0.0000e+00],
        [ 3.9236e-44,  0.0000e+00, -1.4419e+01,  4.5766e-41,  1.4013e-45,
          4.5766e-41,  1.4013e-45],
        [ 1.8470e+31,  1.4013e-45,  2.9427e-44,  3.5032e-44,  2.9427e-44,
          6.9868e-35,  3.0775e-41]])

In [142]:
one_hot_smoothing2.fill_(smoothing_value / (7 - 2))

tensor([[0.0200, 0.0200, 0.0200, 0.0200, 0.0200, 0.0200, 0.0200],
        [0.0200, 0.0200, 0.0200, 0.0200, 0.0200, 0.0200, 0.0200],
        [0.0200, 0.0200, 0.0200, 0.0200, 0.0200, 0.0200, 0.0200],
        [0.0200, 0.0200, 0.0200, 0.0200, 0.0200, 0.0200, 0.0200]])

In [143]:
one_hot_smoothing2.scatter_(1, target.data.unsqueeze(1), 1. - smoothing_value)

tensor([[0.0200, 0.9000, 0.0200, 0.0200, 0.0200, 0.0200, 0.0200],
        [0.0200, 0.0200, 0.0200, 0.0200, 0.9000, 0.0200, 0.0200],
        [0.0200, 0.0200, 0.0200, 0.9000, 0.0200, 0.0200, 0.0200],
        [0.0200, 0.0200, 0.0200, 0.9000, 0.0200, 0.0200, 0.0200]])

In [144]:
input2 = torch.rand((batch_size, tgt_vocab_size))
input2

tensor([[0.4798, 0.7551, 0.9597, 0.0752, 0.7422, 0.7434, 0.2026],
        [0.9297, 0.3580, 0.9226, 0.6550, 0.1149, 0.3633, 0.3856],
        [0.1226, 0.6131, 0.6585, 0.8673, 0.9857, 0.9856, 0.1894],
        [0.0175, 0.2836, 0.0773, 0.6193, 0.0162, 0.1845, 0.8570]])

In [145]:
lsm = F.log_softmax(input2, -1)
lsm

tensor([[-2.0744, -1.7991, -1.5945, -2.4790, -1.8119, -1.8108, -2.3516],
        [-1.5909, -2.1626, -1.5979, -1.8655, -2.4057, -2.1573, -2.1350],
        [-2.5061, -2.0156, -1.9703, -1.7614, -1.6431, -1.6432, -2.4394],
        [-2.2709, -2.0047, -2.2110, -1.6690, -2.2721, -2.1038, -1.4314]])

In [146]:
loss2 = -(one_hot_smoothing2 * lsm).sum(-1)
loss2

tensor([1.8617, 2.3953, 1.8296, 1.7480])

In [147]:
loss2.sum(), loss2.mean()

(tensor(7.8346), tensor(1.9586))

In [ ]:
.empty(size=(배치사이즈, n_classes), device=targets.device) # Returns a tensor filled with uninitialized data
.fill_(smoothing / (n_classes - 1)) # 스무싱값 채우기
.scatter_(1, targets.data.unsqueeze(1), 1. - smoothing) # 

In [169]:
one_hot_smoothing3 = torch.empty(size=(batch_size, sentence_size, tgt_vocab_size))
one_hot_smoothing3

tensor([[[ 1.1479e-41,  0.0000e+00,  7.0625e-43,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  6.9919e-35],
         [ 3.0775e-41,  1.4013e-45,  0.0000e+00,  6.9921e-35,  3.0775e-41,
           1.4013e-45,  0.0000e+00],
         [ 6.9919e-35,  3.0775e-41,  3.9236e-44,  3.0775e-41,  2.8718e-22,
           4.5766e-41,  2.8026e-45],
         [ 4.5766e-41,  2.8026e-45,  4.5766e-41,  1.4013e-45,  0.0000e+00,
           3.9236e-44,  0.0000e+00],
         [-1.4419e+01,  4.5766e-41,  1.4013e-45,  4.5766e-41,  1.4013e-45,
           0.0000e+00,  1.4013e-45],
         [ 2.9427e-44,  3.5032e-44,  2.9427e-44,  6.9919e-35,  3.0775e-41,
          -2.0462e+06,  4.5766e-41],
         [ 1.4013e-45,  4.5766e-41,  1.4013e-45,  2.9427e-44,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  1.4013e-45,  0.0000e+00,  6.9921e-35,  3.0775e-41,
           4.2039e-45,  0.0000e+00],
         [ 6.9920e-35,  3.0775e-41,  6.9920e-35,  3.0775e-41,  6.9921e-35,
           3.0775e-41,  3.923

In [170]:
one_hot_smoothing3.fill_(smoothing_value / (7 - 2))

tensor([[[0.0200, 0.0200, 0.0200, 0.0200, 0.0200, 0.0200, 0.0200],
         [0.0200, 0.0200, 0.0200, 0.0200, 0.0200, 0.0200, 0.0200],
         [0.0200, 0.0200, 0.0200, 0.0200, 0.0200, 0.0200, 0.0200],
         [0.0200, 0.0200, 0.0200, 0.0200, 0.0200, 0.0200, 0.0200],
         [0.0200, 0.0200, 0.0200, 0.0200, 0.0200, 0.0200, 0.0200],
         [0.0200, 0.0200, 0.0200, 0.0200, 0.0200, 0.0200, 0.0200],
         [0.0200, 0.0200, 0.0200, 0.0200, 0.0200, 0.0200, 0.0200],
         [0.0200, 0.0200, 0.0200, 0.0200, 0.0200, 0.0200, 0.0200],
         [0.0200, 0.0200, 0.0200, 0.0200, 0.0200, 0.0200, 0.0200],
         [0.0200, 0.0200, 0.0200, 0.0200, 0.0200, 0.0200, 0.0200]],

        [[0.0200, 0.0200, 0.0200, 0.0200, 0.0200, 0.0200, 0.0200],
         [0.0200, 0.0200, 0.0200, 0.0200, 0.0200, 0.0200, 0.0200],
         [0.0200, 0.0200, 0.0200, 0.0200, 0.0200, 0.0200, 0.0200],
         [0.0200, 0.0200, 0.0200, 0.0200, 0.0200, 0.0200, 0.0200],
         [0.0200, 0.0200, 0.0200, 0.0200, 0.0200, 0.0200, 0.

In [176]:
# 임의로 설정한 target2
target2 = torch.randint(1, 7, (batch_size, sentence_size))
target2

tensor([[3, 6, 2, 2, 1, 3, 6, 5, 3, 1],
        [2, 2, 3, 3, 4, 3, 5, 2, 2, 5],
        [5, 6, 4, 3, 2, 4, 1, 1, 4, 5],
        [6, 2, 6, 5, 4, 2, 5, 6, 3, 4]])

In [6]:
target2.data.unsqueeze(-1)

NameError: name 'target2' is not defined

In [180]:
one_hot_smoothing3.shape

torch.Size([4, 10, 7])

In [190]:
one_hot_smoothing3.scatter_(-1, target2.data.unsqueeze(-1), 1. - smoothing_value)

tensor([[[0.0200, 0.0200, 0.0200, 0.9000, 0.0200, 0.0200, 0.0200],
         [0.0200, 0.0200, 0.0200, 0.0200, 0.0200, 0.0200, 0.9000],
         [0.0200, 0.0200, 0.9000, 0.0200, 0.0200, 0.0200, 0.0200],
         [0.0200, 0.0200, 0.9000, 0.0200, 0.0200, 0.0200, 0.0200],
         [0.0200, 0.9000, 0.0200, 0.0200, 0.0200, 0.0200, 0.0200],
         [0.0200, 0.0200, 0.0200, 0.9000, 0.0200, 0.0200, 0.0200],
         [0.0200, 0.0200, 0.0200, 0.0200, 0.0200, 0.0200, 0.9000],
         [0.0200, 0.0200, 0.0200, 0.0200, 0.0200, 0.9000, 0.0200],
         [0.0200, 0.0200, 0.0200, 0.9000, 0.0200, 0.0200, 0.0200],
         [0.0200, 0.9000, 0.0200, 0.0200, 0.0200, 0.0200, 0.0200]],

        [[0.0200, 0.0200, 0.9000, 0.0200, 0.0200, 0.0200, 0.0200],
         [0.0200, 0.0200, 0.9000, 0.0200, 0.0200, 0.0200, 0.0200],
         [0.0200, 0.0200, 0.0200, 0.9000, 0.0200, 0.0200, 0.0200],
         [0.0200, 0.0200, 0.0200, 0.9000, 0.0200, 0.0200, 0.0200],
         [0.0200, 0.0200, 0.0200, 0.0200, 0.9000, 0.0200, 0.

In [191]:
input3 = torch.rand((batch_size, sentence_size, tgt_vocab_size))
input3

tensor([[[0.5487, 0.9717, 0.2875, 0.6378, 0.3798, 0.6251, 0.0547],
         [0.7149, 0.6334, 0.2612, 0.4684, 0.4854, 0.8011, 0.4887],
         [0.0387, 0.7028, 0.3883, 0.4006, 0.0918, 0.9622, 0.0714],
         [0.2497, 0.6634, 0.4704, 0.9193, 0.6834, 0.4758, 0.1633],
         [0.8740, 0.2523, 0.7250, 0.8226, 0.8815, 0.0855, 0.6467],
         [0.1503, 0.1988, 0.8766, 0.7593, 0.5198, 0.0115, 0.2395],
         [0.4910, 0.6331, 0.8914, 0.3337, 0.6730, 0.4047, 0.5865],
         [0.8565, 0.9654, 0.2573, 0.3034, 0.0878, 0.3548, 0.9704],
         [0.4272, 0.9506, 0.5214, 0.0863, 0.4009, 0.6624, 0.9607],
         [0.7273, 0.2726, 0.2690, 0.9050, 0.8579, 0.6589, 0.1345]],

        [[0.8537, 0.2266, 0.4101, 0.7861, 0.6663, 0.6928, 0.6434],
         [0.4854, 0.0454, 0.5276, 0.5244, 0.9286, 0.8998, 0.8000],
         [0.2681, 0.8655, 0.3118, 0.9855, 0.9043, 0.3197, 0.2840],
         [0.4315, 0.2057, 0.1980, 0.8653, 0.9962, 0.5771, 0.2604],
         [0.6168, 0.0961, 0.2448, 0.3927, 0.6982, 0.5850, 0.

In [193]:
lsm3 = F.log_softmax(input3, -1)
lsm3

tensor([[[-1.9350, -1.5120, -2.1962, -1.8459, -2.1039, -1.8586, -2.4290],
         [-1.7953, -1.8768, -2.2489, -2.0418, -2.0247, -1.7091, -2.0215],
         [-2.3421, -1.6780, -1.9925, -1.9802, -2.2890, -1.4187, -2.3094],
         [-2.2435, -1.8298, -2.0229, -1.5739, -1.8099, -2.0175, -2.3300],
         [-1.7240, -2.3458, -1.8731, -1.7754, -1.7166, -2.5125, -1.9514],
         [-2.2371, -2.1886, -1.5108, -1.6281, -1.8677, -2.3760, -2.1479],
         [-2.0434, -1.9013, -1.6430, -2.2007, -1.8614, -2.1297, -1.9479],
         [-1.6918, -1.5829, -2.2910, -2.2449, -2.4605, -2.1935, -1.5779],
         [-2.1332, -1.6098, -2.0390, -2.4741, -2.1595, -1.8980, -1.5997],
         [-1.8063, -2.2610, -2.2646, -1.6285, -1.6756, -1.8747, -2.3991]],

        [[-1.7230, -2.3501, -2.1666, -1.7906, -1.9104, -1.8839, -1.9333],
         [-2.1002, -2.5401, -2.0580, -2.0612, -1.6570, -1.6858, -1.7856],
         [-2.2894, -1.6920, -2.2458, -1.5720, -1.6532, -2.2379, -2.2735],
         [-2.0658, -2.2916, -2.2992,

In [194]:
loss3 = -(one_hot_smoothing3 * lsm3).sum(-1)
loss3

tensor([[1.9020, 2.0532, 2.0336, 2.0567, 2.3423, 1.7119, 1.9887, 2.2111, 2.4555,
         2.2679],
        [2.1818, 2.0888, 1.6627, 1.7151, 1.8443, 1.7847, 2.1084, 2.2663, 2.2919,
         2.0020],
        [1.9888, 1.8155, 2.0862, 1.8588, 1.7768, 2.3271, 1.7996, 2.0030, 2.1884,
         2.4355],
        [2.1193, 1.9604, 1.8524, 1.8447, 1.6844, 1.6135, 2.0165, 2.2135, 1.9841,
         2.0070]])

In [195]:
loss3.sum(), loss3.mean()

(tensor(80.5441), tensor(2.0136))

In [2]:
!sh satrn12.sh

Traceback (most recent call last):
  File "train.py", line 24, in <module>
    from utils import get_network, get_optimizer
  File "/opt/ml/code/utils.py", line 4, in <module>
    from networks.SATRN import SATRN
  File "/opt/ml/code/networks/SATRN.py", line 665
    num_classes=len(train_dataset.id_to_token)
               ^
SyntaxError: invalid syntax
